## Results 

One of possibilities for improvment we worked on was to predict income of each client in the next month using history of transactions as features. To do that we changed output of baseline model to be real numbers and used MSELoss to train model. Result of predictions can be viewed in the last cell

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install --upgrade numpy pandas tqdm torch catalyst==20.09

Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.19.2)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.1.2)
Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.49.0)
Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.6.0+cu101)
Requirement already up-to-date: catalyst==20.09 in /usr/local/lib/python3.6/dist-packages (20.9)


In [1]:
import os
import json
import pickle
from bisect import bisect_left, bisect_right
from datetime import datetime, timedelta
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
from tqdm import tqdm

# GPU hack if you need
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
os.listdir("./sample_data")

['anscombe.json',
 'README.md',
 'mnist_test.csv',
 'mnist_train_small.csv',
 'california_housing_train.csv',
 'california_housing_test.csv']

# Data

Columns
- `party_rk` – client unique identifier
- `account_rk` – client account unique identifier
- `financial_account_type_cd` – debit/credit card flag
- `transaction_dttm` – operation datetime
- `transaction_type_desc` – purchase/payment/...
- `transaction_amt_rur` – transaction price
- `merchant_type` - DUTY FREE STORES/FUEL DEALERS/RESTAURANTS/ etc
- `merchant_group_rk` - McDonald's/Wildberries/ etc

It's important that table is already sorted by `transaction_dttm` column!!!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATADIR = "/content/drive/My Drive" # "./data"
transactions_path = f"{DATADIR}/avk_hackathon_data_transactions.csv"
to_merge = pd.read_csv(f"{DATADIR}/avk_hackathon_data_transactions.csv")

In [ ]:
df.head(100000)

,Unnamed: 0,Unnamed: 0.1,party_rk,account_rk,financial_account_type_cd,transaction_dttm,transaction_type_desc,transaction_amt_rur,merchant_rk,merchant_type,merchant_group_rk,category,month,cur_month,balance_chng
0,0,0,20337,19666,1,2019-01-01,Покупка,84.00,88676.0,348.0,NaN,Сувениры,2019-01,2019-01-31,15000.0
1,868408,868408,8510,7191,1,2019-01-01,Покупка,17.90,667802.0,286.0,979.0,Супермаркеты,2019-01,2019-01-31,-10000.0
2,557568,557568,14371,28929,2,2019-01-01,Покупка,3064.00,278395.0,286.0,356.0,Супермаркеты,2019-01,2019-01-31,-5000.0
3,868410,868410,8510,7191,1,2019-01-01,Покупка,1011.69,672519.0,297.0,212.0,Топливо,2019-01,2019-01-31,-10000.0
4,557567,557567,14371,28929,2,2019-01-01,Покупка,224.00,611123.0,330.0,NaN,Фаст Фуд,2019-01,2019-01-31,-5000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,904723,904723,62336,71591,2,2019-01-05,Покупка,130.00,1061002.0,291.0,NaN,Супермаркеты,2019-01,2019-01-31,-10000.0
99996,455158,455158,41272,41841,2,2019-01-05,Покупка,105.00,291942.0,284.0,NaN,Разные товары,2019-01,2019-01-31,0.0
99997,885223,885223,38779,38705,1,2019-01-05,Платеж,5099.00,623671.0,1.0,NaN,NaN,2019-01,2019-01-31,0.0
99998,164920,164920,41167,41755,2,2019-01-05,Покупка,259.18,56980.0,286.0,878.0,Супермаркеты,2019-01,2019-01-31,-15000.0


In [ ]:
to_merge['month'] = to_merge['transaction_dttm'].map(lambda x: x[:7])

In [ ]:
merged = pd.merge( left = to_merge, right = f, left_on = ['party_rk','month'], right_on = ['party_rk','month'] )

In [ ]:
merged = df.sort_values(by = 'transaction_dttm', )

In [ ]:
merged.to_csv("/content/drive/My Drive/avk_hackathon_data_account_total_data.csv")

,balance_chng
0,15000.0
1,-10000.0
2,-5000.0
3,-10000.0
4,-5000.0
...,...
11587770,5000.0
11587771,15000.0
11587772,15000.0
11587773,15000.0


In [2]:
DATADIR = "./" # "./data"
transactions_path = f"{DATADIR}/avk_hackathon_data_account_total_data.csv"
#df = pd.read_csv(f"{DATADIR}/avk_hackathon_data_account_total_data.csv")

## Mappings
~1 min

In [19]:
# Prepare & save mappings
mappings = defaultdict(dict)
unk_token = "<UNK>"


def create_mapping(values):
    mapping = {unk_token: 0}
    for v in values:
        if not pd.isna(v):
            mapping[str(v)] = len(mapping)

    return mapping


for col in tqdm(
    [
        "transaction_type_desc",
        "merchant_rk",
        "merchant_type",
        "merchant_group_rk",
        "category",
        "financial_account_type_cd",
        "balance_chng"
    ]
):

    col_values = (
        pd.read_csv(transactions_path, usecols=[col])[col]
        .fillna(unk_token)
        .astype(str)
    )
    mappings[col] = create_mapping(col_values.unique())
    del col_values


with open(f"{DATADIR}/mappings.json", "w") as f:
    json.dump(mappings, f)

100%|██████████| 7/7 [01:57<00:00, 16.83s/it]


In [3]:
with open(f"{DATADIR}/mappings.json", 'r') as f:
     mappings = json.load(f)

## Parse transactions by users
~ 40 min

In [ ]:
# Prepare & save client data
party2dates = defaultdict(list)  # for each party save a series of the transaction dates 
party2sum = defaultdict(list)  # for each party save a series of the transaction costs 
party2merchant_type = defaultdict(list)  # for each party save a series of the transaction_type 
party2trans_type = defaultdict(list)
                                      # for each party save a series of the transaction merchant_type

usecols = [
    "party_rk",
    "transaction_dttm",
    "transaction_amt_rur",
    "merchant_type",
    "transaction_type_desc",
    
]

for chunk in tqdm(
    pd.read_csv(transactions_path, usecols=usecols, chunksize=100_000)
):

    chunk["merchant_type"] = (
        chunk["merchant_type"].fillna(unk_token).astype(str)
    )
    chunk["transaction_type_desc"] = (
        chunk["transaction_type_desc"].fillna(unk_token).astype(str)
    )
    chunk["transaction_amt_rur"] = chunk["transaction_amt_rur"].fillna(0)

    for i, row in chunk.iterrows():
        party2dates[row.party_rk].append(row.transaction_dttm)
        party2sum[row.party_rk].append(row.transaction_amt_rur)
        party2merchant_type[row.party_rk].append(
            mappings["merchant_type"][row.merchant_type]
        )
        party2trans_type[row.party_rk].append(
            mappings["transaction_type_desc"][row.transaction_type_desc]
        )

    del chunk

pickle.dump(party2dates, open(f"{DATADIR}/party2dates.pkl", "wb"))
pickle.dump(party2sum, open(f"{DATADIR}/party2sum.pkl", "wb"))
pickle.dump(party2merchant_type, open(f"{DATADIR}/party2merchant_type.pkl", "wb"))
pickle.dump(party2trans_type, open(f"{DATADIR}/party2trans_type.pkl", "wb"))

116it [24:04, 12.46s/it]


In [4]:
party2dates = pickle.load(open(f"{DATADIR}/party2dates-2.pkl", "rb"))
party2sum = pickle.load(open(f"{DATADIR}/party2sum-2.pkl", "rb"))
party2merchant_type = pickle.load(open(f"{DATADIR}/party2merchant_type-2.pkl", "rb"))
party2trans_type = pickle.load(open(f"{DATADIR}/party2trans_type-2.pkl", "rb"))

In [ ]:
# Prepare & save client data
party2balance = defaultdict(list)  # for each party save a series of the transaction dates 


usecols = [
    "party_rk",
    "balance_chng"

]

for chunk in tqdm(
    pd.read_csv(transactions_path, usecols=usecols, chunksize=100_000)
):
   
    
    chunk["balance_chng"] = chunk["balance_chng"].fillna(0)

    for i, row in chunk.iterrows():
        
        party2balance[row.party_rk].append(row.balance_chng
        )

    del chunk

pickle.dump(party2balance, open(f"{DATADIR}/party2balance.pkl", "wb"))


116it [11:16,  5.83s/it]


In [5]:
party2balance = pickle.load(open(f"{DATADIR}party2balance.pkl", "rb"))

In [ ]:
# load client data
# party2dates = pickle.load(open(f"{DATADIR}/party2dates.pkl", 'rb'))
# party2sum = pickle.load(open(f"{DATADIR}/party2sum.pkl", 'rb'))
# party2merchant_type = pickle.load(open(f"{DATADIR}/party2merchant_type.pkl", 'rb'))
# party2trans_type = pickle.load(open(f"{DATADIR}/party2trans_type.pkl", 'rb'))

In [ ]:
party2balance

## PyTorch dataset

In [97]:
from sklearn.model_selection import train_test_split

train_party, valid_party = train_test_split(
    pd.read_csv(transactions_path, usecols=['party_rk']).party_rk.unique(), 
    train_size=0.8, random_state=42
)

print(f'Train: {len(train_party)} Val: {len(valid_party)}')

Train: 39557 Val: 9890


In [98]:
predict_period_len = 60  # -- days
train_predict_dates = (
    pd.date_range("2019-03-01", "2019-10-31", freq="MS")
    .strftime("%Y-%m-%d")
    .tolist()
)
valid_predict_dates = (
    pd.date_range("2019-11-01", "2019-12-31", freq="MS")
    .strftime("%Y-%m-%d")
    .tolist()
)
submission_predict_dates = (
    pd.date_range("2020-01-01", "2020-02-28", freq="2MS")
    .strftime("%Y-%m-%d")
    .tolist()
)

In [99]:
def prepare_data(party_list, mode="train"):
    """
    This function define the pipeline of the creation of train and valid samples.
    We consider each client from party_list. For each client take each 
    predict_period_start from predict_dates list. All client transaction before
    this date is our features. Next, we look at the customer's transactions in 
    the next two months. This transactions should be predicted. It will form 
    our labels vector.
    """

    data_sum = []
    data_trans_type = []
    data_merchant_type = []
    data_labels = []
    data_income = []
    for party_rk in tqdm(party_list):
        date_series = party2dates[party_rk]
        sum_series = party2sum[party_rk]
        merch_type_series = party2merchant_type[party_rk]
        trans_type_series = party2trans_type[party_rk]
        balance_series = party2balance[party_rk]

        if mode == "train":
            predict_dates = train_predict_dates
        elif mode == "valid":
            predict_dates = valid_predict_dates
        elif mode == "submission":
            predict_dates = submission_predict_dates
        else:
            raise Exception("Unknown mode")

        for predict_period_start in predict_dates:

            predict_period_end = datetime.strftime(
                datetime.strptime(predict_period_start, "%Y-%m-%d")
                + timedelta(days=predict_period_len),
                "%Y-%m-%d",
            )

            l, r = (
                bisect_left(date_series, predict_period_start),
                bisect_right(date_series, predict_period_end),
            )

            history_merch_type = merch_type_series[:l]
            history_sum = sum_series[:l]
            history_trans_type = trans_type_series[:l]
            history_balance = balance_series[:l]
            predict_merch = merch_type_series[l:r]
            predict_balance = balance_series[l:r]

            if predict_merch and l or mode not in ("train", "valid"):
                data_sum.append(history_sum)
                data_trans_type.append(history_trans_type)
                data_merchant_type.append(history_merch_type)
                data_income.append(history_balance)
                data_labels.append(predict_balance)

    return data_sum, data_trans_type, data_merchant_type, data_income, data_labels

In [100]:
train_sum, train_trans_type, train_merchant_type, train_income, train_labels = prepare_data(
    train_party, mode="train"
)
valid_sum, valid_trans_type, valid_merchant_type, valid_income, valid_labels = prepare_data(
    valid_party, mode="valid"
)





  0%|          | 0/39557 [00:00<?, ?it/s]



  1%|          | 330/39557 [00:00<00:11, 3298.96it/s]



  2%|▏         | 663/39557 [00:00<00:11, 3305.96it/s]



  2%|▏         | 971/39557 [00:00<00:11, 3232.19it/s]



  3%|▎         | 1319/39557 [00:00<00:11, 3300.11it/s]



  4%|▍         | 1658/39557 [00:00<00:11, 3325.39it/s]



  5%|▌         | 2004/39557 [00:00<00:11, 3360.25it/s]



  6%|▌         | 2332/39557 [00:00<00:11, 3334.64it/s]



  7%|▋         | 2664/39557 [00:00<00:11, 3291.46it/s]



  8%|▊         | 3003/39557 [00:00<00:11, 3320.38it/s]



  8%|▊         | 3352/39557 [00:01<00:10, 3362.04it/s]



  9%|▉         | 3698/39557 [00:01<00:10, 3390.58it/s]



 10%|█         | 4031/39557 [00:01<00:10, 3370.02it/s]



 11%|█         | 4364/39557 [00:01<00:10, 3347.12it/s]



 12%|█▏        | 4709/39557 [00:01<00:10, 3377.11it/s]



 13%|█▎        | 5069/39557 [00:01<00:10, 3440.48it/s]



 14%|█▎        | 5427/39557 [00:01<00:09, 3477.79it/s]



 15%|█▍        | 5783/3955

In [69]:
#normalize labels
min = 0
max = 0
for i in train_labels:
  for j in i:
    if j>max:
      max=j
    if j<min:
      min = j

In [70]:
for i in range(len(train_labels)):
  for j in range(len(train_labels[i])):
    train_labels[i][j] = np.log(1+(train_labels[i][j]-min))

In [71]:
for i in range(len(valid_labels )):
  for j in range(len(valid_labels [i])):
    valid_labels [i][j] = np.log(1+(valid_labels [i][j]-min))

In [11]:
for i in range(len(train_labels)):
  for j in range(len(train_labels[i])):
    train_labels[i][j] = (train_labels[i][j]-min)/(max-min)

In [12]:
for i in range(len(valid_labels )):
  for j in range(len(valid_labels [i])):
    valid_labels [i][j] = (valid_labels [i][j]-min)/(max-min)


In [101]:
# Third transformation
r=0
mean = 0
for i in range(len(train_labels)):
  for j in range(len(train_labels[i])):
    r+=1
    mean += train_labels[i][j]
mean = mean/r
disp = 0
for i in range(len(train_labels)):
  for j in range(len(train_labels[i])):
    disp += (train_labels[i][j]-mean)**2
disp = disp/(r-1)
for i in range(len(train_labels)):
  for j in range(len(train_labels[i])):
    train_labels[i][j] = (train_labels[i][j] - mean)/np.sqrt(disp)
    
for i in range(len(valid_labels )):
  for j in range(len(valid_labels [i])):
    valid_labels[i][j] = (valid_labels[i][j] - mean)/np.sqrt(disp)

## PyTorch loaders

In [102]:
import torch
from torch.utils.data import Dataset, DataLoader

In [103]:
MERCH_TYPE_NCLASSES = len(mappings['merchant_type'])
TRANS_TYPE_NCLASSES = len(mappings['transaction_type_desc'])
PADDING_LEN = 300

In [104]:
class RSDataset(Dataset):
    def __init__(self, data_sum, data_trans_type, data_merchant_type, labels):
        super(RSDataset, self).__init__()
        self.data_sum = data_sum
        self.data_trans_type = data_trans_type
        self.data_merchant_type = data_merchant_type
        self.labels = labels

    def __len__(self):
        return len(self.data_sum)

    def __getitem__(self, idx):
        targets = torch.tensor(self.labels[idx][0].reshape(-1), dtype = torch.float32)

        item = {
            "features": {},
            "targets": targets,
        }

        sum_feature = np.array(self.data_sum[idx][-PADDING_LEN:])
        sum_feature = np.vectorize(lambda s: np.log(1 + s))(sum_feature)
        if sum_feature.shape[0] < PADDING_LEN:
            pad = np.zeros(
                (PADDING_LEN - sum_feature.shape[0],), dtype=np.float32
            )
            sum_feature = np.hstack((sum_feature, pad))
        item["features"]["sum"] = torch.from_numpy(sum_feature).float()
        
        for feature_name, feature_values in zip(
            ["trans_type", "merchant_type"],
            [self.data_trans_type[idx], self.data_merchant_type[idx]],
        ):

            feature_values = np.array(feature_values[-PADDING_LEN:])
            mask = np.ones(feature_values.shape[0], dtype=np.float32)
            if feature_values.shape[0] < PADDING_LEN:
                feature_values = np.append(
                    feature_values,
                    np.zeros(
                        PADDING_LEN - feature_values.shape[0], dtype=np.int64
                    ),
                )
                mask = np.append(
                    mask,
                    np.zeros(PADDING_LEN - mask.shape[0], dtype=np.float32),
                )
            item["features"][feature_name] = torch.from_numpy(feature_values).long()
            item["features"][f"{feature_name}_mask"] = torch.from_numpy(mask).float()

        return item

In [105]:
train_dataset = RSDataset(
    train_sum, train_trans_type, train_merchant_type, train_labels
)
valid_dataset = RSDataset(
    valid_sum, valid_trans_type, valid_merchant_type, valid_labels
)

In [106]:
train_loader = DataLoader(
    train_dataset, batch_size=64, shuffle=True, num_workers=2
)
valid_loader = DataLoader(
    valid_dataset, batch_size=64, shuffle=False, num_workers=2
)

## Model

This is the baseline model for predicting purchases in `merchant_type` in the next 2 months

In [107]:
import torch.nn as nn
from collections import OrderedDict

In [108]:
params = {
    'merchant_type_emb_dim': 64,
    'trans_type_embedding': 3,
    'transformer_nhead': 2,
    'transformer_dim_feedforward': 256,
    'transformer_dropout': 0.1,
    'dense_unit': 256,
    'num_layers': 4,
}

In [109]:
MERCH_TYPE_NCLASSES, TRANS_TYPE_NCLASSES

(458, 5)

In [110]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.merchant_type_embedding = nn.Embedding(
            MERCH_TYPE_NCLASSES, params["merchant_type_emb_dim"]
        )
        self.trans_type_embedding = nn.Embedding(
            TRANS_TYPE_NCLASSES, params["trans_type_embedding"]
        )

        embedding_size = (
            params["merchant_type_emb_dim"]
            + params["trans_type_embedding"]
            + 1
        )

        transformer_blocks = []
        for i in range(params["num_layers"]):
            transformer_block = nn.TransformerEncoderLayer(
                d_model=embedding_size,
                nhead=params["transformer_nhead"],
                dim_feedforward=params["transformer_dim_feedforward"],
                dropout=params["transformer_dropout"],
            )
            transformer_blocks.append(
                (f"transformer_block_{i}", transformer_block)
            )

        self.transformer_encoder = nn.Sequential(
            OrderedDict(transformer_blocks)
        )

        self.linear = nn.Linear(
            in_features=embedding_size, out_features=params["dense_unit"]
        )
        self.scorer = nn.Linear(
            in_features=params["dense_unit"],
            out_features=1,
        )

    def forward(self, features):

        merchant_type_emb = self.merchant_type_embedding(features["merchant_type"])
        trans_type_emb = self.trans_type_embedding(features["trans_type"])

        merchant_type_emb = merchant_type_emb * features["merchant_type_mask"].unsqueeze(-1)
        trans_type_emb = trans_type_emb * features["trans_type_mask"].unsqueeze(-1)

        embeddings = torch.cat(
            (merchant_type_emb, trans_type_emb, features["sum"].unsqueeze(-1)),
            dim=-1,
        )
      
        transformer_output = self.transformer_encoder(embeddings)
        pooling = torch.mean(transformer_output, dim=1)
        linear = torch.tanh(self.linear(pooling))

        value = self.scorer(linear)

        return value

### One-batch-check

In [111]:
model = Model()
criterion = nn.MSELoss()
batch = next(iter(train_loader))
output = model(batch['features'])
print(output.shape, batch['targets'].shape)
loss = criterion(output,batch['targets'])
print(loss)

torch.Size([64, 1]) torch.Size([64, 1])
tensor(0.0694, grad_fn=<MseLossBackward>)


## Train loop with [Catalyst](https://github.com/catalyst-team/catalyst)

[A comprehensive step-by-step guide to basic and advanced features](https://github.com/catalyst-team/catalyst#step-by-step-guide).

---



In [112]:
from catalyst import dl, utils
from catalyst.utils import metrics

## Custom metrics for this hackathon

In [113]:
from typing import List, Optional, Sequence, Tuple, Union

import numpy as np
import torch
from catalyst.utils.metrics.functional import preprocess_multi_label_metrics
from catalyst.utils.torch import get_activation_fn


def multi_label_metrics(
    outputs: torch.Tensor,
    targets: torch.Tensor,
    threshold: Union[float, torch.Tensor],
    activation: Optional[str] = None,
    eps: float = 1e-7,
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Computes multi-label precision for the specified activation and threshold.

    Args:
        outputs (torch.Tensor): NxK tensor that for each of the N examples
            indicates the probability of the example belonging to each of
            the K classes, according to the model.
        targets (torch.Tensor): binary NxK tensort that encodes which of the K
            classes are associated with the N-th input
            (eg: a row [0, 1, 0, 1] indicates that the example is
            associated with classes 2 and 4)
        threshold (float): threshold for for model output
        activation (str): activation to use for model output
        eps (float): epsilon to avoid zero division
    
    Extended version of 
        https://github.com/catalyst-team/catalyst/blob/master/catalyst/utils/metrics/accuracy.py#L58

    Returns:
        computed multi-label metrics
    """
    outputs, targets, _ = preprocess_multi_label_metrics(
        outputs=outputs, targets=targets
    )
    activation_fn = get_activation_fn(activation)
    outputs = activation_fn(outputs)

    outputs = (outputs > threshold).long()

    accuracy = (targets.long() == outputs.long()).sum().float() / np.prod(
        targets.shape
    )

    intersection = (outputs.long() * targets.long()).sum(axis=1).float()
    num_predicted = outputs.long().sum(axis=1).float()
    num_relevant = targets.long().sum(axis=1).float()
    union = num_predicted + num_relevant

    # Precision = ({predicted items} && {relevant items}) / {predicted items}
    precision = intersection / (num_predicted + eps * (num_predicted == 0))
    # Recall = ({predicted items} && {relevant items}) / {relevant items}
    recall = intersection / (num_relevant + eps * (num_relevant == 0))
    # IoU = ({predicted items} && {relevant items}) / ({predicted items} || {relevant items})
    iou = (intersection + eps * (union == 0)) / (union - intersection + eps)

    return accuracy, precision.mean(), recall.mean(), iou.mean()


def precision_at_k(
    actual: torch.Tensor, 
    predicted: torch.Tensor, 
    k: int,
):
    """
    Computes precision at cutoff k for one sample

    Args:
       actual: (torch.Tensor): tensor of length K with predicted item_ids sorted by relevance
       predicted (torch.Tensor): binary tensor that encodes which of the K
           classes are associated with the N-th input
           (eg: a row [0, 1, 0, 1] indicates that the example is
           associated with classes 2 and 4)
       k (int): parameter k of precison@k

    Returns:
       Computed value of precision@k for given sample
    """
    p_at_k = 0.0
    for item in predicted[:k]:
        if actual[item]:
            p_at_k += 1
    p_at_k /= k

    return p_at_k


def average_precision_at_k(
    actual: torch.Tensor, 
    predicted: torch.Tensor, 
    k: int,
) -> float:
    """
    Computes average precision at cutoff k for one sample

    Args:
      actual: (torch.Tensor): tensor of length K with predicted item_ids sorted by relevance
      predicted (torch.Tensor): binary tensor that encodes which of the K
          classes are associated with the N-th input
          (eg: a row [0, 1, 0, 1] indicates that the example is
          associated with classes 2 and 4)
      k (int): parameter k of AP@k

    Returns:
        Computed value of AP@k for given sample
    """
    ap_at_k = 0.0
    for idx, item in enumerate(predicted[:k]):
        if actual[item]:
            ap_at_k += precision_at_k(actual, predicted, k=idx + 1)
    ap_at_k /= min(k, actual.sum().cpu().numpy())
    

    return ap_at_k


def mean_average_precision_at_k(
    output: torch.Tensor, target: torch.Tensor, top_k: Tuple[int, ...] = (1,)
) -> List[float]:
    """
    Computes mean_average_precision_at_k at set of cutoff parameters K

    Args:
       outputs (torch.Tensor): NxK tensor that for each of the N examples
           indicates the probability of the example belonging to each of
           the K classes, according to the model.
       targets (torch.Tensor): binary NxK tensort that encodes which of the K
           classes are associated with the N-th input
           (eg: a row [0, 1, 0, 1] indicates that the example is
           associated with classes 2 and 4)
       top_k (tuple): list of parameters k at which map@k will be computed


    Returns:
       List of computed values of map@k at each cutoff k from topk
    """
    max_k = max(top_k)
    batch_size = target.size(0)

    _, top_indices = output.topk(k=max_k, dim=1, largest=True, sorted=True)

    result = []
    for k in top_k:  # loop over k
        map_at_k = 0.0
        for actual_target, predicted_items in zip(
            target, top_indices
        ):  # loop over samples
            map_at_k += average_precision_at_k(
                actual_target, predicted_items, k
            )
        map_at_k = map_at_k / batch_size
        result.append(map_at_k)

    return result

In [114]:
# What is Runner?
# https://catalyst-team.github.io/catalyst/api/core.html#runner
class CustomRunner(dl.Runner):

    def _handle_batch(self, batch):
        # model train/valid step
        features, targets = batch["features"], batch["targets"]
        logits = self.model(features)
        scores = torch.sigmoid(logits)

        loss = self.criterion(logits, targets)#(torch.log(1+logits), torch.log(1+targets))

        batch_metrics = {
            "loss": loss
        }
        
        self.input = {"features": features, "targets": targets}
        self.output = {"logits": logits, "scores": scores}
        self.batch_metrics.update(batch_metrics)

        if self.is_train_loader:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
    
    def predict_batch(self, batch):
        # model inference step
        batch = utils.maybe_recursive_call(batch, "to", device=self.device)
        logits = self.model(batch["features"])
        scores = torch.sigmoid(logits)
        return scores

In [115]:
model = Model()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

loaders = {"train": train_loader, "valid": valid_loader}

In [116]:
%load_ext tensorboard
%tensorboard --logdir ./logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 3651820), started 2:03:02 ago. (Use '!kill 3651820' to kill it.)

In [117]:
# For other minimal examples, please follow the link below
# https://github.com/catalyst-team/catalyst#minimal-examples
runner = CustomRunner()
# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=None,
    loaders=loaders,
    logdir="./logs",
    num_epochs=3,
    verbose=True,
    
    load_best_on_end=True,
    overfit=False,  #  <<<--- DO NOT FORGET TO MAKE IT ``False`` 
                    #  (``True`` uses only one batch to check pipeline correctness)
    callbacks=[
        # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html
        # dl.AveragePrecisionCallback(input_key="targets", output_key="scores", prefix="ap"),
        # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
        # dl.AUCCallback(input_key="targets", output_key="scores", prefix="auc"),
    ],
    main_metric="loss", # "ap/mean", 
    minimize_metric=True,
)





1/3 * Epoch (train):   0% 0/3657 [00:00<?, ?it/s]



1/3 * Epoch (train):   0% 0/3657 [00:01<?, ?it/s, loss=0.165]



1/3 * Epoch (train):   0% 1/3657 [00:01<1:07:26,  1.11s/it, loss=0.165]



1/3 * Epoch (train):   0% 1/3657 [00:01<1:07:26,  1.11s/it, loss=1.074]



1/3 * Epoch (train):   0% 2/3657 [00:01<53:06,  1.15it/s, loss=1.074]  



1/3 * Epoch (train):   0% 2/3657 [00:01<53:06,  1.15it/s, loss=0.238]



1/3 * Epoch (train):   0% 3/3657 [00:01<42:58,  1.42it/s, loss=0.238]



1/3 * Epoch (train):   0% 3/3657 [00:02<42:58,  1.42it/s, loss=0.284]



1/3 * Epoch (train):   0% 4/3657 [00:02<35:43,  1.70it/s, loss=0.284]



1/3 * Epoch (train):   0% 4/3657 [00:02<35:43,  1.70it/s, loss=0.243]



1/3 * Epoch (train):   0% 5/3657 [00:02<30:42,  1.98it/s, loss=0.243]



1/3 * Epoch (train):   0% 5/3657 [00:02<30:42,  1.98it/s, loss=0.233]



1/3 * Epoch (train):   0% 6/3657 [00:02<27:24,  2.22it/s, loss=0.233]



1/3 * Epoch (train):   0% 6/3657 [00:03<27:24,  2.22it/s, loss=0.125]

1/3 * Epoch (train):   1% 47/3657 [00:15<18:17,  3.29it/s, loss=0.047]



1/3 * Epoch (train):   1% 48/3657 [00:15<18:06,  3.32it/s, loss=0.047]



1/3 * Epoch (train):   1% 48/3657 [00:16<18:06,  3.32it/s, loss=0.105]



1/3 * Epoch (train):   1% 49/3657 [00:16<18:05,  3.32it/s, loss=0.105]



1/3 * Epoch (train):   1% 49/3657 [00:16<18:05,  3.32it/s, loss=0.464]



1/3 * Epoch (train):   1% 50/3657 [00:16<18:09,  3.31it/s, loss=0.464]



1/3 * Epoch (train):   1% 50/3657 [00:16<18:09,  3.31it/s, loss=0.263]



1/3 * Epoch (train):   1% 51/3657 [00:16<18:01,  3.33it/s, loss=0.263]



1/3 * Epoch (train):   1% 51/3657 [00:16<18:01,  3.33it/s, loss=0.089]



1/3 * Epoch (train):   1% 52/3657 [00:16<17:48,  3.37it/s, loss=0.089]



1/3 * Epoch (train):   1% 52/3657 [00:17<17:48,  3.37it/s, loss=0.273]



1/3 * Epoch (train):   1% 53/3657 [00:17<17:26,  3.44it/s, loss=0.273]



1/3 * Epoch (train):   1% 53/3657 [00:17<17:26,  3.44it/s, loss=0.152]



1/3 * Epoch (train):   1% 54/3657 [00:

1/3 * Epoch (train):   3% 95/3657 [00:29<17:37,  3.37it/s, loss=0.076]



1/3 * Epoch (train):   3% 95/3657 [00:29<17:37,  3.37it/s, loss=0.058]



1/3 * Epoch (train):   3% 96/3657 [00:29<17:39,  3.36it/s, loss=0.058]



1/3 * Epoch (train):   3% 96/3657 [00:30<17:39,  3.36it/s, loss=0.075]



1/3 * Epoch (train):   3% 97/3657 [00:30<17:22,  3.41it/s, loss=0.075]



1/3 * Epoch (train):   3% 97/3657 [00:30<17:22,  3.41it/s, loss=0.078]



1/3 * Epoch (train):   3% 98/3657 [00:30<17:35,  3.37it/s, loss=0.078]



1/3 * Epoch (train):   3% 98/3657 [00:30<17:35,  3.37it/s, loss=0.127]



1/3 * Epoch (train):   3% 99/3657 [00:30<17:53,  3.31it/s, loss=0.127]



1/3 * Epoch (train):   3% 99/3657 [00:31<17:53,  3.31it/s, loss=0.053]



1/3 * Epoch (train):   3% 100/3657 [00:31<18:19,  3.23it/s, loss=0.053]



1/3 * Epoch (train):   3% 100/3657 [00:31<18:19,  3.23it/s, loss=3.354]



1/3 * Epoch (train):   3% 101/3657 [00:31<18:04,  3.28it/s, loss=3.354]



1/3 * Epoch (train):   3% 101/3657 

1/3 * Epoch (train):   4% 142/3657 [00:42<16:25,  3.57it/s, loss=0.291]



1/3 * Epoch (train):   4% 142/3657 [00:43<16:25,  3.57it/s, loss=0.116]



1/3 * Epoch (train):   4% 143/3657 [00:43<16:15,  3.60it/s, loss=0.116]



1/3 * Epoch (train):   4% 143/3657 [00:43<16:15,  3.60it/s, loss=0.069]



1/3 * Epoch (train):   4% 144/3657 [00:43<16:02,  3.65it/s, loss=0.069]



1/3 * Epoch (train):   4% 144/3657 [00:43<16:02,  3.65it/s, loss=0.252]



1/3 * Epoch (train):   4% 145/3657 [00:43<15:57,  3.67it/s, loss=0.252]



1/3 * Epoch (train):   4% 145/3657 [00:44<15:57,  3.67it/s, loss=0.732]



1/3 * Epoch (train):   4% 146/3657 [00:44<15:51,  3.69it/s, loss=0.732]



1/3 * Epoch (train):   4% 146/3657 [00:44<15:51,  3.69it/s, loss=0.101]



1/3 * Epoch (train):   4% 147/3657 [00:44<15:52,  3.69it/s, loss=0.101]



1/3 * Epoch (train):   4% 147/3657 [00:44<15:52,  3.69it/s, loss=0.119]



1/3 * Epoch (train):   4% 148/3657 [00:44<15:46,  3.71it/s, loss=0.119]



1/3 * Epoch (train):   4%

1/3 * Epoch (train):   5% 189/3657 [00:56<16:40,  3.47it/s, loss=0.031]



1/3 * Epoch (train):   5% 189/3657 [00:56<16:40,  3.47it/s, loss=0.180]



1/3 * Epoch (train):   5% 190/3657 [00:56<16:12,  3.57it/s, loss=0.180]



1/3 * Epoch (train):   5% 190/3657 [00:56<16:12,  3.57it/s, loss=0.808]



1/3 * Epoch (train):   5% 191/3657 [00:56<16:01,  3.60it/s, loss=0.808]



1/3 * Epoch (train):   5% 191/3657 [00:57<16:01,  3.60it/s, loss=0.052]



1/3 * Epoch (train):   5% 192/3657 [00:57<15:55,  3.63it/s, loss=0.052]



1/3 * Epoch (train):   5% 192/3657 [00:57<15:55,  3.63it/s, loss=0.189]



1/3 * Epoch (train):   5% 193/3657 [00:57<15:45,  3.67it/s, loss=0.189]



1/3 * Epoch (train):   5% 193/3657 [00:57<15:45,  3.67it/s, loss=0.070]



1/3 * Epoch (train):   5% 194/3657 [00:57<15:38,  3.69it/s, loss=0.070]



1/3 * Epoch (train):   5% 194/3657 [00:57<15:38,  3.69it/s, loss=0.515]



1/3 * Epoch (train):   5% 195/3657 [00:57<15:30,  3.72it/s, loss=0.515]



1/3 * Epoch (train):   5%

...

2/3 * Epoch (train):   4% 161/3657 [00:49<17:45,  3.28it/s, loss=0.053]



2/3 * Epoch (train):   4% 162/3657 [00:49<17:31,  3.32it/s, loss=0.053]



2/3 * Epoch (train):   4% 162/3657 [00:49<17:31,  3.32it/s, loss=0.134]



2/3 * Epoch (train):   4% 163/3657 [00:49<17:18,  3.36it/s, loss=0.134]



2/3 * Epoch (train):   4% 163/3657 [00:50<17:18,  3.36it/s, loss=0.242]



2/3 * Epoch (train):   4% 164/3657 [00:50<17:17,  3.37it/s, loss=0.242]



2/3 * Epoch (train):   4% 164/3657 [00:50<17:17,  3.37it/s, loss=0.295]



2/3 * Epoch (train):   5% 165/3657 [00:50<17:18,  3.36it/s, loss=0.295]



2/3 * Epoch (train):   5% 165/3657 [00:50<17:18,  3.36it/s, loss=0.075]



2/3 * Epoch (train):   5% 166/3657 [00:50<17:06,  3.40it/s, loss=0.075]



2/3 * Epoch (train):   5% 166/3657 [00:50<17:06,  3.40it/s, loss=0.121]



2/3 * Epoch (train):   5% 167/3657 [00:50<17:07,  3.40it/s, loss=0.121]



2/3 * Epoch (train):   5% 167/3657 [00:51<17:07,  3.40it/s, loss=0.131]



2/3 * Epoch (train):   5%

2/3 * Epoch (train):   6% 208/3657 [01:04<19:28,  2.95it/s, loss=0.183]



2/3 * Epoch (train):   6% 209/3657 [01:04<19:08,  3.00it/s, loss=0.183]



2/3 * Epoch (train):   6% 209/3657 [01:04<19:08,  3.00it/s, loss=0.353]



2/3 * Epoch (train):   6% 210/3657 [01:04<18:52,  3.04it/s, loss=0.353]



2/3 * Epoch (train):   6% 210/3657 [01:05<18:52,  3.04it/s, loss=0.018]



2/3 * Epoch (train):   6% 211/3657 [01:05<18:41,  3.07it/s, loss=0.018]



2/3 * Epoch (train):   6% 211/3657 [01:05<18:41,  3.07it/s, loss=0.319]



2/3 * Epoch (train):   6% 212/3657 [01:05<18:35,  3.09it/s, loss=0.319]



2/3 * Epoch (train):   6% 212/3657 [01:05<18:35,  3.09it/s, loss=66.340]



2/3 * Epoch (train):   6% 213/3657 [01:05<18:37,  3.08it/s, loss=66.340]



2/3 * Epoch (train):   6% 213/3657 [01:06<18:37,  3.08it/s, loss=0.103] 



2/3 * Epoch (train):   6% 214/3657 [01:06<18:50,  3.05it/s, loss=0.103]



2/3 * Epoch (train):   6% 214/3657 [01:06<18:50,  3.05it/s, loss=0.042]



2/3 * Epoch (train):  

2/3 * Epoch (train):   7% 255/3657 [01:19<15:39,  3.62it/s, loss=0.049]



2/3 * Epoch (train):   7% 256/3657 [01:19<15:44,  3.60it/s, loss=0.049]



2/3 * Epoch (train):   7% 256/3657 [01:19<15:44,  3.60it/s, loss=0.089]



2/3 * Epoch (train):   7% 257/3657 [01:19<15:39,  3.62it/s, loss=0.089]



2/3 * Epoch (train):   7% 257/3657 [01:19<15:39,  3.62it/s, loss=0.186]



2/3 * Epoch (train):   7% 258/3657 [01:19<16:03,  3.53it/s, loss=0.186]



2/3 * Epoch (train):   7% 258/3657 [01:20<16:03,  3.53it/s, loss=0.100]



2/3 * Epoch (train):   7% 259/3657 [01:20<16:26,  3.45it/s, loss=0.100]



2/3 * Epoch (train):   7% 259/3657 [01:20<16:26,  3.45it/s, loss=0.088]



2/3 * Epoch (train):   7% 260/3657 [01:20<16:21,  3.46it/s, loss=0.088]



2/3 * Epoch (train):   7% 260/3657 [01:20<16:21,  3.46it/s, loss=0.035]



2/3 * Epoch (train):   7% 261/3657 [01:20<16:31,  3.43it/s, loss=0.035]



2/3 * Epoch (train):   7% 261/3657 [01:21<16:31,  3.43it/s, loss=0.072]



2/3 * Epoch (train):   7%

2/3 * Epoch (train):   8% 302/3657 [01:32<14:54,  3.75it/s, loss=0.086]



2/3 * Epoch (train):   8% 303/3657 [01:32<14:53,  3.75it/s, loss=0.086]



2/3 * Epoch (train):   8% 303/3657 [01:32<14:53,  3.75it/s, loss=0.053]



2/3 * Epoch (train):   8% 304/3657 [01:32<14:49,  3.77it/s, loss=0.053]



2/3 * Epoch (train):   8% 304/3657 [01:32<14:49,  3.77it/s, loss=0.876]



2/3 * Epoch (train):   8% 305/3657 [01:32<14:51,  3.76it/s, loss=0.876]



2/3 * Epoch (train):   8% 305/3657 [01:33<14:51,  3.76it/s, loss=0.091]



2/3 * Epoch (train):   8% 306/3657 [01:33<14:53,  3.75it/s, loss=0.091]



2/3 * Epoch (train):   8% 306/3657 [01:33<14:53,  3.75it/s, loss=0.044]



2/3 * Epoch (train):   8% 307/3657 [01:33<14:41,  3.80it/s, loss=0.044]



2/3 * Epoch (train):   8% 307/3657 [01:33<14:41,  3.80it/s, loss=0.150]



2/3 * Epoch (train):   8% 308/3657 [01:33<14:42,  3.79it/s, loss=0.150]



2/3 * Epoch (train):   8% 308/3657 [01:34<14:42,  3.79it/s, loss=0.121]



2/3 * Epoch (train):   8%

2/3 * Epoch (train):  10% 349/3657 [01:45<14:48,  3.72it/s, loss=0.229]



2/3 * Epoch (train):  10% 350/3657 [01:45<14:43,  3.74it/s, loss=0.229]



2/3 * Epoch (train):  10% 350/3657 [01:46<14:43,  3.74it/s, loss=0.064]



2/3 * Epoch (train):  10% 351/3657 [01:46<14:38,  3.76it/s, loss=0.064]



2/3 * Epoch (train):  10% 351/3657 [01:46<14:38,  3.76it/s, loss=0.067]



2/3 * Epoch (train):  10% 352/3657 [01:46<14:35,  3.77it/s, loss=0.067]



2/3 * Epoch (train):  10% 352/3657 [01:46<14:35,  3.77it/s, loss=0.131]



2/3 * Epoch (train):  10% 353/3657 [01:46<14:35,  3.77it/s, loss=0.131]



2/3 * Epoch (train):  10% 353/3657 [01:46<14:35,  3.77it/s, loss=0.153]



2/3 * Epoch (train):  10% 354/3657 [01:46<14:35,  3.77it/s, loss=0.153]



2/3 * Epoch (train):  10% 354/3657 [01:47<14:35,  3.77it/s, loss=0.088]



2/3 * Epoch (train):  10% 355/3657 [01:47<14:33,  3.78it/s, loss=0.088]



2/3 * Epoch (train):  10% 355/3657 [01:47<14:33,  3.78it/s, loss=0.123]



2/3 * Epoch (train):  10%

In [119]:
torch.save(model.state_dict(), './model_balance.pth')

In [120]:
# Results
model = Model()
model.load_state_dict(torch.load( f"{DATADIR}model_balance.pth"))
criterion = nn.MSELoss()
batch = next(iter(valid_loader))
output = model(batch['features'])

loss = criterion(output,batch['targets'])
print("Loss: ", loss)
print("Prediction: ", output*np.sqrt(disp)+mean)

Loss:  tensor(0.1382, grad_fn=<MseLossBackward>)
Prediction:  tensor([[  8530.9229],
        [  2237.5354],
        [ 32236.1836],
        [ 32738.2812],
        [ 31670.2070],
        [ 29873.0117],
        [ 39716.1992],
        [ 38400.8828],
        [ 26392.9277],
        [ 26533.6309],
        [ 35061.2695],
        [ 32983.9922],
        [-30158.9863],
        [-30803.9121],
        [  -620.5317],
        [-19605.6328],
        [ 35765.7969],
        [ 33961.1523],
        [ 37802.1328],
        [ 35757.5078],
        [-11247.3545],
        [-12581.0283],
        [ -2200.2246],
        [ -6601.7285],
        [-15263.5850],
        [-15113.0977],
        [ 30507.1055],
        [ 29300.2227],
        [ 41693.0898],
        [ 40304.7617],
        [ 35598.0000],
        [ 36432.2188],
        [-15704.2803],
        [-14870.8125],
        [ -9172.7266],
        [ -8689.7510],
        [ 41606.6641],
        [ 39472.2461],
        [ 41683.7500],
        [ 42309.9258],
        [ 16385.37